### import 

In [175]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar

def save_dataset_to_pkl (dir, file_name, sheet_name=0) :
    """폴더경로, 파일명(확장자x), [쉬트네임 또는 순번]을 주면
    조건에 맞는 칼럼은 문자열로 읽고,
    빈칸은 빈문자열로 읽어 pkl로 저장해주는 함수
    """
    # 확장자
    read_ext = ".xlsx"
    write_ext = ".pkl"
    # 문자열로 읽을 컬럼명 패턴
    pattern = re.compile('(키|코드|계좌)$')
    # 
    path_file = join(dir, file_name)
    dtype_mapping = {col: str for col in pd.read_excel(path_file + read_ext, sheet_name=sheet_name).columns if pattern.search(col)}
    df = pd.read_excel(path_file + read_ext, sheet_name=sheet_name, dtype=dtype_mapping).fillna("")
    # 아래는 read_excel을 3번 호출해서 더 비효율적이라네?
    # df = pd.read_excel(path_file + read_ext, sheet_name=sheet_name, dtype={col: str for col in pd.read_excel(path_file + read_ext, sheet_name=sheet_name).columns if pattern.search(col)}).fillna("")
    df.to_pickle(path_file + write_ext)

def months_between_dates(start_date, end_date):
    # 월말은 알아서 1.0개월 차이로 계산해줌
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    delta = relativedelta(end_date, start_date)
    months = delta.years * 12 + delta.months

    return months, delta.days

wd = r'D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\check\개인회생'

### 엑셀읽어 pkl 저장, 읽기

In [2]:
files = ("개인회생새창_20230807_1112_진행중_계좌키분리_기준데이터","입금조회새창_20230808_0841_개인회생_100101~230807")

In [3]:
# pkl 저장
for f in files :
    save_dataset_to_pkl(wd, f)

In [3]:
### pkl 다시 읽기
rehabilitation_ori = pd.read_pickle(join(wd, files[0]+".pkl"))
deposit_ori = pd.read_pickle(join(wd, files[1]+".pkl"))

### 입금이상 - 개인회생파일

In [235]:
# 인가건
rehabilitation_인가 = rehabilitation_ori[rehabilitation_ori["인가/미인가"]=="인가"].copy()
# rehabilitation_인가 = rehabilitation_ori[rehabilitation_ori.채무자키=="20410785"].copy()
# # # 매각건 구분
# # reha_sell = rehabilitation_인가[rehabilitation_인가.채무상태.str.contains("매각|DS")|rehabilitation_인가.담당자.str.contains("매각|DS")].copy()
# # reha_own = rehabilitation_인가[~(rehabilitation_인가.채무상태.str.contains("매각|DS")|rehabilitation_인가.담당자.str.contains("매각|DS"))].copy()

In [236]:
# 납입회차에 따라 입금되었어야 할 금액 계산
입금이상유무,회차입금총액,분납시작회차,분납시작기준일, 분납시작기준 = [], [], [], [], []

for i, v in tqdm(rehabilitation_인가.iterrows(), total=len(rehabilitation_인가.index)) : 
    # 한계좌에 대한 회차별 변제액 표 작성
    회차별변제액 = pd.DataFrame(0, index = range(0,62), columns=["변제예정액", "비고"])
    마이너스있음 = False
    
    # try :
    for n in range(1, 14) : # 최대구간은 13까지임
        구간 = str(v["변제회차"+str(n)]) # 문자열
        구간변제액 = v["변제예정액"+str(n)] # int64
        
        if 구간 == "" : # 이전구간에서 최종회차 나옴. 또는 변제조회가 안 됨. 
            break
        elif re.search("-", 구간) : # 별도로 처리하자.
            마이너스있음 = True
            break
        else :
            if re.search("~", 구간) :
                구간시작회차,구간끝회차 = 구간.split("~")
                구간시작회차 = int(구간시작회차)
                구간끝회차 = int(구간끝회차)
            else : 
                구간시작회차 = int(구간)
                구간끝회차 = 구간시작회차

            # 변제회차별 회차정보 및 변제금액 넣기
            회차별변제액.loc[구간시작회차:구간끝회차,"변제예정액"] = 구간변제액

    # except : 
    #     print(f"분납키 : {v.분납키}")
    #     print(f"구간 : {구간, type(구간)}")
        
    
    # 최초회차 구하기
    # 1) 컷오프일 잡기
    if v.매입전상태 == "인가" : 
        if v.분납컷오프일 == "" : 
            종합컷오프일 = v.컷오프일
            기준 = "계좌컷오프일"
        else :
            종합컷오프일 = v.분납컷오프일
            기준 = "분납컷오프일"
    else : # 변제 중 매입
        종합컷오프일 = v.분납시작일
        기준 = "분납시작일"

    # 2) 분납시작일과 컷오프일의 차이로 최초회차 구하기
    월차이, 일차이 = months_between_dates(v.분납시작일, 종합컷오프일)
    if (월차이 == 0) & (일차이 == 0) :
        최초회차 = 1
    elif (월차이 == 0) & (일차이 > 0) :
        최초회차 = 1 + 1
    else : # 1.0 이상
        최초회차 = 1 + int(월차이) + 1 # 1.0이든 1.1이든 무조건 1회차 추가 

    # 납입회차
    납입회차 = int(v.납입회차)

    # 계좌별 값 추가
    회차입금액 = 회차별변제액.loc[최초회차:납입회차]["변제예정액"].sum()
    회차입금총액.append(회차입금액)
    if 회차입금액 != v.총분납입금 :
        입금이상유무.append("이상")
    elif 마이너스있음 : # 총액이 다르면 어차피 회차정보 볼테니 
        입금이상유무.append("-회차")
    else :
        입금이상유무.append("")
    분납시작회차.append(최초회차)
    분납시작기준일.append(종합컷오프일)
    분납시작기준.append(기준)
    

rehabilitation_인가["입금이상유무"] = 입금이상유무
rehabilitation_인가["회차입금총액"] = 회차입금총액
rehabilitation_인가["분납시작회차"] = 분납시작회차
rehabilitation_인가["분납시작기준일"] = 분납시작기준일
rehabilitation_인가["분납시작기준"] = 분납시작기준
cond = rehabilitation_인가["채무상태"].str.contains("매각|DS") | rehabilitation_인가["담당자"].str.contains("매각|DS")
rehabilitation_인가["매각여부"] = np.where(cond,"매각", "보유")

100%|██████████| 6238/6238 [00:02<00:00, 2734.33it/s]


In [238]:
result = rehabilitation_인가[rehabilitation_인가.입금이상유무 != ""][["채무자키","계좌키","분납키","매각사구분","분납자관계","주민번호인","분납자성명인",
    "매각여부","인가/미인가","현재결과","분납상태","총분납입금","회차입금총액","입금이상유무","초과입금","납입회차","현재변제회차", "분납시작회차","분납시작기준일", "분납시작기준",
    "변제회차1", "변제예정액1","변제회차2", "변제예정액2","변제회차3", "변제예정액3","변제회차4", "변제예정액4","변제회차5", "변제예정액5", 
    "반영OPB","총분납금","분납미납금","분납잔금","총분납잔금", "총분납회차","현재회차","잔여회차","미납회차","전체변제회차","잔여변제회차",
    "관할법원","사건번호","접수일","개시결정일","변제계획인가일","변제현황조회","신고조회결과","입금조회결과","입금요청결과","변제채권번호입력","변제채권번호"]]
result.rename(columns={"납입회차":"납입회차(전산)", "현재변제회차":"납입회차(법원)"}, inplace=True)
# 주민번호기준 정렬
result.sort_values("주민번호인", inplace=True)

In [240]:
result.to_excel(join(wd, "check_개인회생_입금이상1.xlsx"), index=False)

### 입금이상 - 입금파일에서

In [380]:
# 개인회생 입금건 
deposit_reha = deposit_ori[deposit_ori.입금구분=="개인회생"][["채무자키","계좌키","주민번호인","입금자구분","입금자","입금합계","입금일","등록파일","채무상태","담당자"]].copy()
# 매각 표시
cond = deposit_reha["채무상태"].str.contains("매각|DS") | deposit_reha["담당자"].str.contains("매각|DS")
deposit_reha["매각여부"] = np.where(cond,"매각", "")
deposit_reha.drop(["채무상태","담당자"],axis=1,inplace=True)


# 개인회생 계좌별,입금자별 그룹화
#deposit_reha_grouped = deposit_reha.groupby(["채무자키","계좌키",'입금자'])

# 개인회생 파일
rehabilitation_인가 = rehabilitation_ori[rehabilitation_ori["인가/미인가"]=="인가"].copy()
rehabilitation_인가 = rehabilitation_인가.rename(columns={"분납자성명인":"입금자"})

# 변제예정액 모음 만들기
def collect_nonzero_values(row):
    return [value for value in row if value not in (0, "")]
변제예정액 = rehabilitation_인가[["변제예정액"+str(i) for i in range(1,14)]].copy()
rehabilitation_인가["변제예정액모음"] = 변제예정액.apply(collect_nonzero_values, axis=1)

# # 필요칼럼만 모으고, 개인회생 계좌별,입금자별 그룹화
rehabilitation_인가_grouped = rehabilitation_인가[["채무자키","계좌키","입금자","변제예정액모음"]]

# 입금파일에 변제예정액모음열 병합하기
merged_df = deposit_reha.merge(rehabilitation_인가_grouped, on=["채무자키","계좌키", "입금자"], how="left")

In [381]:
# 변제예정액값이 리스트가 아니거나(NaN), 빈리스트인경우를 제외
merged_df2 = merged_df[merged_df["변제예정액모음"].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)].copy()

In [382]:
def check_error(row):
    if any(row["입금합계"] % amount == 0 for amount in row["변제예정액모음"]):
        return ""
    else : 
        return "오류"

merged_df2["배수여부"] = merged_df2.apply(check_error, axis=1)


In [383]:
# 저장
merged_df2[merged_df2["배수여부"]=="오류"].sort_values(["채무자키","계좌키","입금자","입금일"]).to_excel(join(wd, "check_개인회생_입금이상_by입금조회.xlsx"), index=False)